# CIViC Evidence Analysis
This notebook contains an analysis on CIViC evidence data

# Analysis of Normalized Queries

In [1]:
from pathlib import Path
from enum import Enum
import zipfile

import pandas as pd
from civicpy import civic as civicpy
import plotly.express as px

In [2]:
# Use latest cache that has been pushed to the repo
#latest_cache_zip_path = sorted(Path().glob("cache-*.pkl.zip"))[-1]
#print(f"Using {latest_cache_zip_path} for civicpy cache")

#with zipfile.ZipFile(latest_cache_zip_path, "r") as zip_ref:
#    zip_ref.extractall()

#civicpy.load_cache(local_cache_path=Path("cache.pkl"), on_stale="ignore")

## Total Variants in CIViC

In [3]:
civic_variant_ids = civicpy.get_all_variants()
total_number_variants = len(civic_variant_ids)
f"Total Number of variants in CIViC: {total_number_variants}"

'Total Number of variants in CIViC: 3581'

## Total Evidence items in CIViC

Need to remove all rejected evidence items

In [4]:
civic_evidence_ids = civicpy.get_all_evidence(include_status=["accepted", "submitted"])

In [5]:
total_ac_sub_evidence = len(civic_evidence_ids)
f"Total Number of accepted and submitted evidence items in CIViC: {total_ac_sub_evidence}"

'Total Number of accepted and submitted evidence items in CIViC: 9914'

## Total Molecular Profiles in CIViC

In [6]:
civic_molprof_ids = civicpy.get_all_molecular_profiles(include_status=["accepted", "submitted", "rejected"])

In [7]:
len(civic_evidence_ids)

9914

# Create analysis functions / global variables 

In [8]:
class VariantNormType(str, Enum):
    """Variation Normalization types"""

    NORMALIZED = "Normalized"
    UNABLE_TO_NORMALIZE = "Unable to Normalize"
    NOT_SUPPORTED = "Not Supported"


VARIANT_NORM_TYPE_VALUES = [v.value for v in VariantNormType.__members__.values()]

In [9]:
class VariantCategory(str, Enum):
    """Create enum for the kind of variants that are in CIViC."""

    EXPRESSION = "Expression"
    EPIGENETIC_MODIFICATION = "Epigenetic Modification"
    FUSION = "Fusion"
    PROTEIN_CONS = "Protein Consequence"
    GENE_FUNC = "Gene Function"
    REARRANGEMENTS = "Rearrangements"
    COPY_NUMBER = "Copy Number"
    OTHER = "Other"
    GENOTYPES_EASY = "Genotypes Easy"
    GENOTYPES_COMPOUND = "Genotypes Compound"
    REGION_DEFINED_VAR = "Region Defined Variant"
    TRANSCRIPT_VAR = "Transcript Variant"  # no attempt to normalize these ones, since there is no query we could use


VARIANT_CATEGORY_VALUES = [v.value for v in VariantCategory.__members__.values()]

## Summary dicts 

These dictionaries will be mutated and used at the end of the analysis

In [10]:
variant_analysis_summary = {
    "Variant Category": VARIANT_NORM_TYPE_VALUES,
    "Count of CIViC Variant Items per Category":[],
    "Fraction of all CIViC Variant Items": [],
    "Percentage of all CIViC Variant Items": [],
    "Fraction of Accepted Variant Items": [],
    "Percentage of Accepted Variant Items": [],
    "Fraction of Not Accepted Variant Items": [],
    "Percentage of Not Accepted Variant Items": []
}
variant_analysis_summary

{'Variant Category': ['Normalized', 'Unable to Normalize', 'Not Supported'],
 'Count of CIViC Variant Items per Category': [],
 'Fraction of all CIViC Variant Items': [],
 'Percentage of all CIViC Variant Items': [],
 'Fraction of Accepted Variant Items': [],
 'Percentage of Accepted Variant Items': [],
 'Fraction of Not Accepted Variant Items': [],
 'Percentage of Not Accepted Variant Items': []}

In [11]:
evidence_analysis_summary = {
    "Variant Category": VARIANT_NORM_TYPE_VALUES,
    "Count of CIViC Evidence Items per Category":[],
    "Fraction of all CIViC Evidence Items": [],
    "Percentage of all CIViC Evidence Items": [],
    "Fraction of Accepted Evidence Items": [],
    "Percentage of Accepted Evidence Items": [],
    "Fraction of Submitted Evidence Items": [],
    "Percentage of Submitted Evidence Items": []
}
evidence_analysis_summary

{'Variant Category': ['Normalized', 'Unable to Normalize', 'Not Supported'],
 'Count of CIViC Evidence Items per Category': [],
 'Fraction of all CIViC Evidence Items': [],
 'Percentage of all CIViC Evidence Items': [],
 'Fraction of Accepted Evidence Items': [],
 'Percentage of Accepted Evidence Items': [],
 'Fraction of Submitted Evidence Items': [],
 'Percentage of Submitted Evidence Items': []}

## Define Analysis Functions

In [12]:
def variant_analysis(df: pd.DataFrame, variant_norm_type: VariantNormType) -> pd.DataFrame:
    """Do variant analysis (counts, percentages)

    :param df: Dataframe of variants
    :param variant_norm_type: The kind of variants that are in `df`
    :return: Transformed dataframe with variant ID duplicates dropped
    """
    # Drop duplicate rows
    df = df.drop_duplicates(subset=["variant_id"])
    variant_ids = list(df["variant_id"])

    # Count
    num_variants = len(variant_ids)
    fraction_variants = f"{num_variants} / {total_number_variants}"
    print(f"\nNumber of {variant_norm_type.value} Variants in CIViC: {fraction_variants}")

    # Percentage
    percentage_variants = f"{num_variants / total_number_variants * 100:.2f}%"
    print(f"Percentage of {variant_norm_type.value} Variants in CIViC: {percentage_variants}")

    # Get accepted counts
    num_accepted_variants = df.variant_accepted.sum()
    fraction_accepted_variants = f"{num_accepted_variants} / {num_variants}"
    print(f"\nNumber of accepted {variant_norm_type.value} Variants: {fraction_accepted_variants}")

    # Get accepted percentage
    percentage_accepted_variants = f"{num_accepted_variants / num_variants * 100:.2f}%"
    print(f"Percentage of accepted {variant_norm_type.value} Variants: {percentage_accepted_variants}")

    # Get not accepted counts
    num_not_accepted_variants = len(df) - num_accepted_variants
    fraction_not_accepted_variants = f"{num_not_accepted_variants} / {num_variants}"
    print(f"\nNumber of not accepted {variant_norm_type.value} Variants: {fraction_not_accepted_variants}")

    # Get not accepted percentage
    percentage_not_accepted_variants = f"{num_not_accepted_variants / num_variants * 100:.2f}%"
    print(f"Percentage of not accepted {variant_norm_type.value} Variants: {percentage_not_accepted_variants}")

    variant_analysis_summary["Count of CIViC Variant Items per Category"].append(num_variants)
    variant_analysis_summary["Fraction of all CIViC Variant Items"].append(fraction_variants)
    variant_analysis_summary["Percentage of all CIViC Variant Items"].append(percentage_variants)
    variant_analysis_summary["Fraction of Accepted Variant Items"].append(fraction_accepted_variants)
    variant_analysis_summary["Percentage of Accepted Variant Items"].append(percentage_accepted_variants)
    variant_analysis_summary["Fraction of Not Accepted Variant Items"].append(fraction_not_accepted_variants)
    variant_analysis_summary["Percentage of Not Accepted Variant Items"].append(percentage_not_accepted_variants)

    return df

In [13]:
def transform_df_evidence_ids(df: pd.DataFrame) -> pd.DataFrame:
    """Transform dataframe to include evidence ID information

    :param df: Dataframe of variants
    :return: Transformed dataframe with evidence ID information
    """
    tmp_df = df.copy(deep=True)

    _variants_evidence_ids = []
    variant_ids = list(tmp_df["variant_id"])

    for v_id in variant_ids:
        _variant_evidence_ids = []

        for variant in civic_variant_ids:
            if int(v_id) == variant.id:
                for mp in variant.molecular_profiles:
                    for e in mp.evidence_items:
                        if e.id not in _variant_evidence_ids:
                            _variant_evidence_ids.append(e.id)

        _variants_evidence_ids.append(_variant_evidence_ids or "")

    tmp_df["evidence_ids"] = _variants_evidence_ids

    # Explode and rename evidence ids field
    tmp_df = tmp_df.explode(column="evidence_ids")
    tmp_df = tmp_df.rename(columns={"evidence_ids": "evidence_id"})

    return tmp_df

In [14]:
def transform_df_evidence(df: pd.DataFrame) -> pd.DataFrame:
    """Transform dataframe to include evidence status, rating, and level

    :param df: Dataframe of variants
    :return: Transformed dataframe with evidence status, rating, and level information.
        Rejected evidence items will be dropped.
    """
    variants_evidence_ids = list(df["evidence_id"])

    # Add evidence status, rating, and level information
    _variants_evidence_statuses = []
    _variants_evidence_ratings = []
    _variants_evidence_levels = []

    for eid in variants_evidence_ids:
        _variant_evidence_statuses = []
        _variant_evidence_ratings = []
        _variant_evidence_levels = []

        for evidence in civic_evidence_ids:
            if eid and (int(eid) == evidence.id):
                if evidence.status not in _variant_evidence_statuses:
                    _variant_evidence_statuses.append(evidence.status)

                if evidence.rating not in _variant_evidence_ratings:
                    _variant_evidence_ratings.append(evidence.rating)

                if evidence.evidence_level not in _variant_evidence_levels:
                    _variant_evidence_levels.append(evidence.evidence_level)
                    
        _variants_evidence_statuses.append(_variant_evidence_statuses or "")
        _variants_evidence_ratings.append(_variant_evidence_ratings or "")
        _variants_evidence_levels.append(_variant_evidence_levels or "")

    df["evidence_status"] = _variants_evidence_statuses
    df["evidence_status"] = df["evidence_status"].str.join(", ")
    df["evidence_rating"] = _variants_evidence_ratings
    df["evidence_level"] = _variants_evidence_levels

    # Drop rejected evidence items
    df = df.drop(df[df.evidence_status == "rejected"].index)

    return df


In [15]:
def evidence_analysis(df: pd.DataFrame, variant_norm_type: VariantNormType) -> pd.DataFrame:
    """Do evidence analysis (counts, percentages)

    :param df: Dataframe of variants
    :param variant_norm_type: The kind of variants that are in `df`
    :return: Transformed dataframe with evidence ID duplicates dropped
    """
    # Count
    num_variant_unique_evidence_items = len(set(df.evidence_id))
    fraction_evidence_items = f"{num_variant_unique_evidence_items} / {total_ac_sub_evidence}"
    print(f"Number of {variant_norm_type.value} Variant Evidence items in CIViC: {fraction_evidence_items}")

    # Percentage
    percentage_evidence_items = f"{num_variant_unique_evidence_items / total_ac_sub_evidence * 100:.2f}%"
    print(f"Percentage of {variant_norm_type.value} Variant Evidence items in CIViC: {percentage_evidence_items}")

    # Add evidence accepted column
    df["evidence_accepted"] = df.evidence_status.map({"accepted": True, "submitted": False})

    # Drop evidence id duplicates
    df = df.drop_duplicates(subset=["evidence_id"])

    # Get accepted counts
    num_accepted_evidences_variants = df.evidence_accepted.sum()
    fraction_accepted_evidences_variants = f"{num_accepted_evidences_variants} / {num_variant_unique_evidence_items}"
    print(f"\nNumber of accepted {variant_norm_type.value} Variant Evidence items: {fraction_accepted_evidences_variants}")

    # Get accepted percentage
    percentage_accepted_evidences_variants = f"{num_accepted_evidences_variants / num_variant_unique_evidence_items * 100:.2f}%"
    print(f"Percentage of accepted {variant_norm_type.value} Variant Evidence items: {percentage_accepted_evidences_variants}")

    # Get submitted counts
    number_submitted_evidences_variants = len(df) - num_accepted_evidences_variants
    fraction_submitted_evidences_variants = f"{number_submitted_evidences_variants} / {num_variant_unique_evidence_items}"
    print(f"\nNumber of submitted {variant_norm_type.value} Variant Evidence items: {fraction_submitted_evidences_variants}")

    # Get submitted percentage
    percentage_submitted_evidences_variants = f"{number_submitted_evidences_variants / num_variant_unique_evidence_items * 100:.2f}%"
    print(f"Percentage of not submitted {variant_norm_type.value} Variant Evidence items: {percentage_submitted_evidences_variants}")

    evidence_analysis_summary["Count of CIViC Evidence Items per Category"].append(num_variant_unique_evidence_items)
    evidence_analysis_summary["Fraction of all CIViC Evidence Items"].append(fraction_evidence_items)
    evidence_analysis_summary["Percentage of all CIViC Evidence Items"].append(percentage_evidence_items)
    evidence_analysis_summary["Fraction of Accepted Evidence Items"].append(fraction_accepted_evidences_variants)
    evidence_analysis_summary["Percentage of Accepted Evidence Items"].append(percentage_accepted_evidences_variants)
    evidence_analysis_summary["Fraction of Submitted Evidence Items"].append(fraction_submitted_evidences_variants)
    evidence_analysis_summary["Percentage of Submitted Evidence Items"].append(percentage_submitted_evidences_variants)

    return df

In [16]:
def transform_df_mp_id(df: pd.DataFrame) -> pd.DataFrame:
    """Transform dataframe to include molecular profile ID information

    :param df: Dataframe of variants
    :return: Transformed dataframe with molecular profile ID information
    """
    tmp_df = df.copy(deep=True)

    variants_molprof_ids = []
    variant_ids = list(tmp_df["variant_id"])

    for v_id in variant_ids:
        variant_molprof_ids = []

        for variant in civic_variant_ids:
            if int(v_id) == variant.id:
                for mp in variant.molecular_profiles:
                    if mp.id not in variant_molprof_ids:
                        variant_molprof_ids.append(mp.id)

        variants_molprof_ids.append(variant_molprof_ids or "")

    tmp_df["molecular_profile_id"] = variants_molprof_ids
    return tmp_df

In [17]:
def transform_df_mp_score(df: pd.DataFrame) -> pd.DataFrame:
    """Transform dataframe to include molecular profile score information

    :param df: Dataframe of variants
    :return: Transformed dataframe with molecular profile score information
    """
    variants_molprof_scores = []
    normalized_variant_molprof_ids = list(df["molecular_profile_id"])

    for mp_ids in normalized_variant_molprof_ids:
        variant_molprof_scores = []
        for mp_id in mp_ids:
            for molprof in civic_molprof_ids:
                if int(mp_id) == molprof.id:
                    if molprof.molecular_profile_score not in variant_molprof_scores:
                        variant_molprof_scores.append(molprof.molecular_profile_score)

        variants_molprof_scores.append(variant_molprof_scores or "")

    df["molecular_profile_score"] = variants_molprof_scores
    return df

In [18]:
def transform_df_mp_score_sum(df: pd.DataFrame) -> pd.DataFrame:
    """Transform dataframe to include molecular profile score sum information

    :param df: Dataframe of variants
    :return: Transformed dataframe with molecular profile score sum information
    """
    df["molecular_profile_score_sum"] = df["molecular_profile_score"].apply(lambda x: sum(x))
    return df

## List of Normalized Variants ID's

In [19]:
normalized_queries_df = pd.read_csv("able_to_normalize_queries.csv", sep="\t")
normalized_queries_df.head()

,variant_id,query,query_type,variant_accepted,vrs_id
0,2489,NC_000003.11:g.10191648_10191649insC,genomic,True,ga4gh:VA.A34ZoIhq4xBuQbcE3bkj29n6diS6RzLB
1,1988,NC_000003.11:g.10191649A>T,genomic,True,ga4gh:VA.JcEpDvhUtgDWU4A-bxqLUuczBNb8QqRf
2,2488,3-10191647-T-G,genomic,True,ga4gh:VA.7nGd8dgHbqtxMHk_rLxrB6_IMAzJ8XnH
3,1986,NC_000003.11:g.10191648G>T,genomic,True,ga4gh:VA.AmLtooLEvgdnEHD5YVWk6u1e2XBe7FiP
4,1987,NC_000003.11:g.10191649A>G,genomic,True,ga4gh:VA.KIz00usFWEmJHNyqmVL61obfgfRPgOIa


## Variant analysis

In [20]:
normalized_queries_df = variant_analysis(normalized_queries_df, VariantNormType.NORMALIZED)
normalized_queries_df.head()


Number of Normalized Variants in CIViC: 1869 / 3581
Percentage of Normalized Variants in CIViC: 52.19%

Number of accepted Normalized Variants: 866 / 1869
Percentage of accepted Normalized Variants: 46.33%

Number of not accepted Normalized Variants: 1003 / 1869
Percentage of not accepted Normalized Variants: 53.67%


,variant_id,query,query_type,variant_accepted,vrs_id
0,2489,NC_000003.11:g.10191648_10191649insC,genomic,True,ga4gh:VA.A34ZoIhq4xBuQbcE3bkj29n6diS6RzLB
1,1988,NC_000003.11:g.10191649A>T,genomic,True,ga4gh:VA.JcEpDvhUtgDWU4A-bxqLUuczBNb8QqRf
2,2488,3-10191647-T-G,genomic,True,ga4gh:VA.7nGd8dgHbqtxMHk_rLxrB6_IMAzJ8XnH
3,1986,NC_000003.11:g.10191648G>T,genomic,True,ga4gh:VA.AmLtooLEvgdnEHD5YVWk6u1e2XBe7FiP
4,1987,NC_000003.11:g.10191649A>G,genomic,True,ga4gh:VA.KIz00usFWEmJHNyqmVL61obfgfRPgOIa


In [21]:
variant_analysis_summary

{'Variant Category': ['Normalized', 'Unable to Normalize', 'Not Supported'],
 'Count of CIViC Variant Items per Category': [1869],
 'Fraction of all CIViC Variant Items': ['1869 / 3581'],
 'Percentage of all CIViC Variant Items': ['52.19%'],
 'Fraction of Accepted Variant Items': ['866 / 1869'],
 'Percentage of Accepted Variant Items': ['46.33%'],
 'Fraction of Not Accepted Variant Items': ['1003 / 1869'],
 'Percentage of Not Accepted Variant Items': ['53.67%']}

## Transform df for evidence analysis

In [22]:
normalized_queries_add_evidence_df = transform_df_evidence_ids(normalized_queries_df)
normalized_queries_add_evidence_df.head()

,variant_id,query,query_type,variant_accepted,vrs_id,evidence_id
0,2489,NC_000003.11:g.10191648_10191649insC,genomic,True,ga4gh:VA.A34ZoIhq4xBuQbcE3bkj29n6diS6RzLB,9347
0,2489,NC_000003.11:g.10191648_10191649insC,genomic,True,ga4gh:VA.A34ZoIhq4xBuQbcE3bkj29n6diS6RzLB,6724
1,1988,NC_000003.11:g.10191649A>T,genomic,True,ga4gh:VA.JcEpDvhUtgDWU4A-bxqLUuczBNb8QqRf,5336
2,2488,3-10191647-T-G,genomic,True,ga4gh:VA.7nGd8dgHbqtxMHk_rLxrB6_IMAzJ8XnH,10779
2,2488,3-10191647-T-G,genomic,True,ga4gh:VA.7nGd8dgHbqtxMHk_rLxrB6_IMAzJ8XnH,6723


In [23]:
normalized_queries_add_evidence_df = transform_df_evidence(normalized_queries_add_evidence_df)
normalized_queries_add_evidence_df.head()

,variant_id,query,query_type,variant_accepted,vrs_id,evidence_id,evidence_status,evidence_rating,evidence_level
0,2489,NC_000003.11:g.10191648_10191649insC,genomic,True,ga4gh:VA.A34ZoIhq4xBuQbcE3bkj29n6diS6RzLB,9347,submitted,[3],[C]
0,2489,NC_000003.11:g.10191648_10191649insC,genomic,True,ga4gh:VA.A34ZoIhq4xBuQbcE3bkj29n6diS6RzLB,6724,accepted,[2],[C]
1,1988,NC_000003.11:g.10191649A>T,genomic,True,ga4gh:VA.JcEpDvhUtgDWU4A-bxqLUuczBNb8QqRf,5336,accepted,[2],[C]
2,2488,3-10191647-T-G,genomic,True,ga4gh:VA.7nGd8dgHbqtxMHk_rLxrB6_IMAzJ8XnH,10779,submitted,[3],[C]
2,2488,3-10191647-T-G,genomic,True,ga4gh:VA.7nGd8dgHbqtxMHk_rLxrB6_IMAzJ8XnH,6723,accepted,[2],[C]


## Evidence analysis

In [24]:
normalized_queries_add_evidence_df = evidence_analysis(normalized_queries_add_evidence_df, VariantNormType.NORMALIZED)
normalized_queries_add_evidence_df.head()

Number of Normalized Variant Evidence items in CIViC: 5960 / 9914
Percentage of Normalized Variant Evidence items in CIViC: 60.12%

Number of accepted Normalized Variant Evidence items: 2033 / 5960
Percentage of accepted Normalized Variant Evidence items: 34.11%

Number of submitted Normalized Variant Evidence items: 3927 / 5960
Percentage of not submitted Normalized Variant Evidence items: 65.89%


,variant_id,query,query_type,variant_accepted,vrs_id,evidence_id,evidence_status,evidence_rating,evidence_level,evidence_accepted
0,2489,NC_000003.11:g.10191648_10191649insC,genomic,True,ga4gh:VA.A34ZoIhq4xBuQbcE3bkj29n6diS6RzLB,9347,submitted,[3],[C],False
0,2489,NC_000003.11:g.10191648_10191649insC,genomic,True,ga4gh:VA.A34ZoIhq4xBuQbcE3bkj29n6diS6RzLB,6724,accepted,[2],[C],True
1,1988,NC_000003.11:g.10191649A>T,genomic,True,ga4gh:VA.JcEpDvhUtgDWU4A-bxqLUuczBNb8QqRf,5336,accepted,[2],[C],True
2,2488,3-10191647-T-G,genomic,True,ga4gh:VA.7nGd8dgHbqtxMHk_rLxrB6_IMAzJ8XnH,10779,submitted,[3],[C],False
2,2488,3-10191647-T-G,genomic,True,ga4gh:VA.7nGd8dgHbqtxMHk_rLxrB6_IMAzJ8XnH,6723,accepted,[2],[C],True


## Impact
molecular profile score

### Import molecular profile id

In [25]:
normalized_queries_add_molprof_df = transform_df_mp_id(normalized_queries_df)
normalized_queries_add_molprof_df.head()

,variant_id,query,query_type,variant_accepted,vrs_id,molecular_profile_id
0,2489,NC_000003.11:g.10191648_10191649insC,genomic,True,ga4gh:VA.A34ZoIhq4xBuQbcE3bkj29n6diS6RzLB,[2362]
1,1988,NC_000003.11:g.10191649A>T,genomic,True,ga4gh:VA.JcEpDvhUtgDWU4A-bxqLUuczBNb8QqRf,[1864]
2,2488,3-10191647-T-G,genomic,True,ga4gh:VA.7nGd8dgHbqtxMHk_rLxrB6_IMAzJ8XnH,[2361]
3,1986,NC_000003.11:g.10191648G>T,genomic,True,ga4gh:VA.AmLtooLEvgdnEHD5YVWk6u1e2XBe7FiP,[1862]
4,1987,NC_000003.11:g.10191649A>G,genomic,True,ga4gh:VA.KIz00usFWEmJHNyqmVL61obfgfRPgOIa,[1863]


In [26]:
normalized_queries_add_molprof_df.loc[normalized_queries_add_molprof_df["variant_id"] == 190]

,variant_id,query,query_type,variant_accepted,vrs_id,molecular_profile_id
82,190,EGFR Amplification,protein,True,ga4gh:CX.sEHT64Lm86QaTXzw39uKLkBUbEkp4h_X,"[190, 4175, 4346, 4567]"


### Import molecular profile scores

In [27]:
normalized_queries_add_molprof_df = transform_df_mp_score(normalized_queries_add_molprof_df)
normalized_queries_add_molprof_df.head()

,variant_id,query,query_type,variant_accepted,vrs_id,molecular_profile_id,molecular_profile_score
0,2489,NC_000003.11:g.10191648_10191649insC,genomic,True,ga4gh:VA.A34ZoIhq4xBuQbcE3bkj29n6diS6RzLB,[2362],[5.0]
1,1988,NC_000003.11:g.10191649A>T,genomic,True,ga4gh:VA.JcEpDvhUtgDWU4A-bxqLUuczBNb8QqRf,[1864],[5.0]
2,2488,3-10191647-T-G,genomic,True,ga4gh:VA.7nGd8dgHbqtxMHk_rLxrB6_IMAzJ8XnH,[2361],[5.0]
3,1986,NC_000003.11:g.10191648G>T,genomic,True,ga4gh:VA.AmLtooLEvgdnEHD5YVWk6u1e2XBe7FiP,[1862],[10.0]
4,1987,NC_000003.11:g.10191649A>G,genomic,True,ga4gh:VA.KIz00usFWEmJHNyqmVL61obfgfRPgOIa,[1863],[5.0]


In [28]:
normalized_queries_add_molprof_df.loc[normalized_queries_add_molprof_df["variant_id"] == 190]

,variant_id,query,query_type,variant_accepted,vrs_id,molecular_profile_id,molecular_profile_score
82,190,EGFR Amplification,protein,True,ga4gh:CX.sEHT64Lm86QaTXzw39uKLkBUbEkp4h_X,"[190, 4175, 4346, 4567]","[173.0, 5.0, 0.0]"


In [29]:
normalized_queries_add_molprof_df = transform_df_mp_score_sum(normalized_queries_add_molprof_df)
normalized_queries_add_molprof_df.head()

,variant_id,query,query_type,variant_accepted,vrs_id,molecular_profile_id,molecular_profile_score,molecular_profile_score_sum
0,2489,NC_000003.11:g.10191648_10191649insC,genomic,True,ga4gh:VA.A34ZoIhq4xBuQbcE3bkj29n6diS6RzLB,[2362],[5.0],5.0
1,1988,NC_000003.11:g.10191649A>T,genomic,True,ga4gh:VA.JcEpDvhUtgDWU4A-bxqLUuczBNb8QqRf,[1864],[5.0],5.0
2,2488,3-10191647-T-G,genomic,True,ga4gh:VA.7nGd8dgHbqtxMHk_rLxrB6_IMAzJ8XnH,[2361],[5.0],5.0
3,1986,NC_000003.11:g.10191648G>T,genomic,True,ga4gh:VA.AmLtooLEvgdnEHD5YVWk6u1e2XBe7FiP,[1862],[10.0],10.0
4,1987,NC_000003.11:g.10191649A>G,genomic,True,ga4gh:VA.KIz00usFWEmJHNyqmVL61obfgfRPgOIa,[1863],[5.0],5.0


In [30]:
normalized_queries_add_molprof_df.loc[normalized_queries_add_molprof_df["variant_id"] == 190]

,variant_id,query,query_type,variant_accepted,vrs_id,molecular_profile_id,molecular_profile_score,molecular_profile_score_sum
82,190,EGFR Amplification,protein,True,ga4gh:CX.sEHT64Lm86QaTXzw39uKLkBUbEkp4h_X,"[190, 4175, 4346, 4567]","[173.0, 5.0, 0.0]",178.0


# Analysis of Unable to Normalize Queries

## List of Unable to Normalize Variant ID's

In [31]:
not_normalized_queries_df = pd.read_csv("unable_to_normalize_queries.csv", sep="\t")
not_normalized_queries_df.head()

,variant_id,query,query_type,variant_accepted,exception_raised,message,warnings
0,748,MLH1 *757L,protein,True,False,unable to normalize,"[""Unable to find valid result for classificati..."
1,3718,AR A748V,protein,False,False,unable to normalize,"[""Unable to find valid result for classificati..."
2,3725,AR A765T,protein,False,False,unable to normalize,"[""Unable to find valid result for classificati..."
3,248,TERT C228T,protein,True,False,unable to normalize,"[""Unable to find valid result for classificati..."
4,4004,TERT C250T,protein,False,False,unable to normalize,"[""Unable to find valid result for classificati..."


## Variant analysis

In [32]:
not_normalized_queries_df = variant_analysis(not_normalized_queries_df, VariantNormType.UNABLE_TO_NORMALIZE)
not_normalized_queries_df.head()


Number of Unable to Normalize Variants in CIViC: 63 / 3581
Percentage of Unable to Normalize Variants in CIViC: 1.76%

Number of accepted Unable to Normalize Variants: 9 / 63
Percentage of accepted Unable to Normalize Variants: 14.29%

Number of not accepted Unable to Normalize Variants: 54 / 63
Percentage of not accepted Unable to Normalize Variants: 85.71%


,variant_id,query,query_type,variant_accepted,exception_raised,message,warnings
0,748,MLH1 *757L,protein,True,False,unable to normalize,"[""Unable to find valid result for classificati..."
1,3718,AR A748V,protein,False,False,unable to normalize,"[""Unable to find valid result for classificati..."
2,3725,AR A765T,protein,False,False,unable to normalize,"[""Unable to find valid result for classificati..."
3,248,TERT C228T,protein,True,False,unable to normalize,"[""Unable to find valid result for classificati..."
4,4004,TERT C250T,protein,False,False,unable to normalize,"[""Unable to find valid result for classificati..."


## Transform df for evidence analysis

In [33]:
not_normalized_quer_add_evidence_df = transform_df_evidence_ids(not_normalized_queries_df)
not_normalized_quer_add_evidence_df.head()

,variant_id,query,query_type,variant_accepted,exception_raised,message,warnings,evidence_id
0,748,MLH1 *757L,protein,True,False,unable to normalize,"[""Unable to find valid result for classificati...",1812
1,3718,AR A748V,protein,False,False,unable to normalize,"[""Unable to find valid result for classificati...",10128
2,3725,AR A765T,protein,False,False,unable to normalize,"[""Unable to find valid result for classificati...",10135
3,248,TERT C228T,protein,True,False,unable to normalize,"[""Unable to find valid result for classificati...",655
3,248,TERT C228T,protein,True,False,unable to normalize,"[""Unable to find valid result for classificati...",1646


In [34]:
not_normalized_quer_add_evidence_df = transform_df_evidence(not_normalized_quer_add_evidence_df)
not_normalized_quer_add_evidence_df.head()

,variant_id,query,query_type,variant_accepted,exception_raised,message,warnings,evidence_id,evidence_status,evidence_rating,evidence_level
0,748,MLH1 *757L,protein,True,False,unable to normalize,"[""Unable to find valid result for classificati...",1812,accepted,[1],[C]
1,3718,AR A748V,protein,False,False,unable to normalize,"[""Unable to find valid result for classificati...",10128,submitted,[3],[D]
2,3725,AR A765T,protein,False,False,unable to normalize,"[""Unable to find valid result for classificati...",10135,submitted,[3],[D]
3,248,TERT C228T,protein,True,False,unable to normalize,"[""Unable to find valid result for classificati...",655,accepted,[5],[B]
3,248,TERT C228T,protein,True,False,unable to normalize,"[""Unable to find valid result for classificati...",1646,accepted,[3],[B]


## Evidence analysis

In [35]:
not_normalized_quer_add_evidence_df = evidence_analysis(not_normalized_quer_add_evidence_df, VariantNormType.UNABLE_TO_NORMALIZE)
not_normalized_quer_add_evidence_df.head()

Number of Unable to Normalize Variant Evidence items in CIViC: 80 / 9914
Percentage of Unable to Normalize Variant Evidence items in CIViC: 0.81%

Number of accepted Unable to Normalize Variant Evidence items: 14 / 80
Percentage of accepted Unable to Normalize Variant Evidence items: 17.50%

Number of submitted Unable to Normalize Variant Evidence items: 66 / 80
Percentage of not submitted Unable to Normalize Variant Evidence items: 82.50%


,variant_id,query,query_type,variant_accepted,exception_raised,message,warnings,evidence_id,evidence_status,evidence_rating,evidence_level,evidence_accepted
0,748,MLH1 *757L,protein,True,False,unable to normalize,"[""Unable to find valid result for classificati...",1812,accepted,[1],[C],True
1,3718,AR A748V,protein,False,False,unable to normalize,"[""Unable to find valid result for classificati...",10128,submitted,[3],[D],False
2,3725,AR A765T,protein,False,False,unable to normalize,"[""Unable to find valid result for classificati...",10135,submitted,[3],[D],False
3,248,TERT C228T,protein,True,False,unable to normalize,"[""Unable to find valid result for classificati...",655,accepted,[5],[B],True
3,248,TERT C228T,protein,True,False,unable to normalize,"[""Unable to find valid result for classificati...",1646,accepted,[3],[B],True


## Impact
molecular profile score

### Import molecular profile id

In [36]:
not_normalized_queries_add_molprof_df = transform_df_mp_id(not_normalized_queries_df)
not_normalized_queries_add_molprof_df.head()

,variant_id,query,query_type,variant_accepted,exception_raised,message,warnings,molecular_profile_id
0,748,MLH1 *757L,protein,True,False,unable to normalize,"[""Unable to find valid result for classificati...",[729]
1,3718,AR A748V,protein,False,False,unable to normalize,"[""Unable to find valid result for classificati...",[3586]
2,3725,AR A765T,protein,False,False,unable to normalize,"[""Unable to find valid result for classificati...",[3593]
3,248,TERT C228T,protein,True,False,unable to normalize,"[""Unable to find valid result for classificati...",[244]
4,4004,TERT C250T,protein,False,False,unable to normalize,"[""Unable to find valid result for classificati...",[3872]


### Import molecular profile scores

In [37]:
not_normalized_queries_add_molprof_df = transform_df_mp_score(not_normalized_queries_add_molprof_df)
not_normalized_queries_add_molprof_df.head()

,variant_id,query,query_type,variant_accepted,exception_raised,message,warnings,molecular_profile_id,molecular_profile_score
0,748,MLH1 *757L,protein,True,False,unable to normalize,"[""Unable to find valid result for classificati...",[729],[2.5]
1,3718,AR A748V,protein,False,False,unable to normalize,"[""Unable to find valid result for classificati...",[3586],[0.0]
2,3725,AR A765T,protein,False,False,unable to normalize,"[""Unable to find valid result for classificati...",[3593],[0.0]
3,248,TERT C228T,protein,True,False,unable to normalize,"[""Unable to find valid result for classificati...",[244],[40.0]
4,4004,TERT C250T,protein,False,False,unable to normalize,"[""Unable to find valid result for classificati...",[3872],[0.0]


In [38]:
not_normalized_queries_add_molprof_df = transform_df_mp_score_sum(not_normalized_queries_add_molprof_df)
not_normalized_queries_add_molprof_df.head()

,variant_id,query,query_type,variant_accepted,exception_raised,message,warnings,molecular_profile_id,molecular_profile_score,molecular_profile_score_sum
0,748,MLH1 *757L,protein,True,False,unable to normalize,"[""Unable to find valid result for classificati...",[729],[2.5],2.5
1,3718,AR A748V,protein,False,False,unable to normalize,"[""Unable to find valid result for classificati...",[3586],[0.0],0.0
2,3725,AR A765T,protein,False,False,unable to normalize,"[""Unable to find valid result for classificati...",[3593],[0.0],0.0
3,248,TERT C228T,protein,True,False,unable to normalize,"[""Unable to find valid result for classificati...",[244],[40.0],40.0
4,4004,TERT C250T,protein,False,False,unable to normalize,"[""Unable to find valid result for classificati...",[3872],[0.0],0.0


In [39]:
not_normalized_queries_add_molprof_df.count()

variant_id                     63
query                          63
query_type                     63
variant_accepted               63
exception_raised               63
message                        63
warnings                       63
molecular_profile_id           63
molecular_profile_score        63
molecular_profile_score_sum    63
dtype: int64

In [40]:
not_normalized_queries_add_molprof_df["molecular_profile_score_sum"].value_counts()

molecular_profile_score_sum
0.0     54
20.0     2
2.5      1
40.0     1
35.0     1
7.5      1
5.0      1
3.0      1
31.0     1
Name: count, dtype: int64

In [41]:
not_normalized_queries_add_molprof_df[not_normalized_queries_add_molprof_df["variant_accepted"]==True].min()

variant_id                                                                   248
query                                                           3-10191481-GAA-C
query_type                                                               genomic
variant_accepted                                                            True
exception_raised                                                           False
message                                                      unable to normalize
warnings                       ["Unable to find valid result for classificati...
molecular_profile_id                                                       [244]
molecular_profile_score                                                    [2.5]
molecular_profile_score_sum                                                  2.5
dtype: object

# Analysis of Not Supported Variants

### List of Not Supported Variant ID's

In [42]:
not_supported_queries_df = pd.read_csv("not_supported_variants.csv", sep="\t")
not_supported_queries_df.head()

,variant_id,gene_name,variant_name,category,variant_accepted
0,4170,VHL,NaN,Transcript Variant,False
1,4417,ALK,FBXO11::ALK,Fusion,False
2,4214,VHL,NaN,Transcript Variant,False
3,4216,VHL,NaN,Transcript Variant,False
4,4278,VHL,NaN,Transcript Variant,False


## Variant Analysis

In [43]:
not_supported_queries_df = variant_analysis(not_supported_queries_df, VariantNormType.NOT_SUPPORTED)
not_supported_queries_df.head()


Number of Not Supported Variants in CIViC: 1622 / 3581
Percentage of Not Supported Variants in CIViC: 45.29%

Number of accepted Not Supported Variants: 788 / 1622
Percentage of accepted Not Supported Variants: 48.58%

Number of not accepted Not Supported Variants: 834 / 1622
Percentage of not accepted Not Supported Variants: 51.42%


,variant_id,gene_name,variant_name,category,variant_accepted
0,4170,VHL,NaN,Transcript Variant,False
1,4417,ALK,FBXO11::ALK,Fusion,False
2,4214,VHL,NaN,Transcript Variant,False
3,4216,VHL,NaN,Transcript Variant,False
4,4278,VHL,NaN,Transcript Variant,False


In [44]:
not_supported_queries_df["variant_accepted"].value_counts()

variant_accepted
False    834
True     788
Name: count, dtype: int64

### Not Supported Variant Analysis by Subcategory 

In [45]:
not_supported_variant_analysis_summary = {
    "Category": VARIANT_CATEGORY_VALUES,
    "Count of CIViC Variant Items per Category":[],
    "Fraction of Not Supported Variant Items": [],
    "Percent of Not Supported Variant Items": [],
    "Fraction of all CIViC Variant Items": [],
    "Percent of all CIViC Variant Items": [],
    "Fraction of Accepted Variant Items": [],
    "Percent of Accepted Variant Items": [],
    "Fraction of Not Accepted Variant Items": [],
    "Percent of Not Accepted Variant Items": []
}

In [46]:
not_supported_variant_categories_summary_data = dict()
total_number_unique_not_supported_variants = len(set(not_supported_queries_df.variant_id))

for category in VARIANT_CATEGORY_VALUES:  # These are not supported categories
    not_supported_variant_categories_summary_data[category] = {}
    category_df = not_supported_queries_df[not_supported_queries_df.category == category]

    # Count
    number_unique_not_supported_category_variants = len(set(category_df.variant_id))
    not_supported_variant_categories_summary_data[category]["number_unique_not_supported_category_variants"] = number_unique_not_supported_category_variants

    # Fraction
    fraction_not_supported_category_variant_of_civic = f"{number_unique_not_supported_category_variants} / {total_number_variants}"
    not_supported_variant_categories_summary_data[category]["fraction_not_supported_category_variant_of_civic"] = fraction_not_supported_category_variant_of_civic

    # Percent
    percent_not_supported_category_variant_of_civic = f"{number_unique_not_supported_category_variants / total_number_variants * 100:.2f}%"
    not_supported_variant_categories_summary_data[category]["percent_not_supported_category_variant_of_civic"] = percent_not_supported_category_variant_of_civic

    # Not supported fraction
    fraction_not_supported_category_variant_of_total_not_supported = f"{number_unique_not_supported_category_variants} / {total_number_unique_not_supported_variants}"
    not_supported_variant_categories_summary_data[category]["fraction_not_supported_category_variant_of_total_not_supported"] = fraction_not_supported_category_variant_of_total_not_supported

    # Not supported percent
    percent_not_supported_category_variant_of_total_not_supported = f"{number_unique_not_supported_category_variants / total_number_unique_not_supported_variants * 100:.2f}%"
    not_supported_variant_categories_summary_data[category]["percent_not_supported_category_variant_of_total_not_supported"] = percent_not_supported_category_variant_of_total_not_supported

    # Accepted fraction
    number_accepted_not_supported_category_variants = category_df.variant_accepted.sum()
    fraction_accepted_not_supported_category_variants = f"{number_accepted_not_supported_category_variants} / {number_unique_not_supported_category_variants}"
    not_supported_variant_categories_summary_data[category]["fraction_accepted_not_supported_category_variants"] = fraction_accepted_not_supported_category_variants

    # Accepted percent
    percentage_accepted_not_supported_category_variants = f"{number_accepted_not_supported_category_variants / number_unique_not_supported_category_variants * 100:.2f}%"
    not_supported_variant_categories_summary_data[category]["percentage_accepted_not_supported_category_variants"] = percentage_accepted_not_supported_category_variants

    # Not accepted fraction
    number_not_accepted_not_supported_category_variants = len(category_df) - number_accepted_not_supported_category_variants
    fraction_not_accepted_not_supported_category_variants = f" {number_not_accepted_not_supported_category_variants} / {number_unique_not_supported_category_variants}"
    not_supported_variant_categories_summary_data[category]["fraction_not_accepted_not_supported_category_variants"] = fraction_not_accepted_not_supported_category_variants

    # Not accepted percent
    percentage_not_accepted_not_supported_category_variants = f"{number_not_accepted_not_supported_category_variants / number_unique_not_supported_category_variants * 100:.2f}%"
    not_supported_variant_categories_summary_data[category]["percentage_not_accepted_not_supported_category_variants"] = percentage_not_accepted_not_supported_category_variants

    not_supported_variant_analysis_summary["Count of CIViC Variant Items per Category"].append(number_unique_not_supported_category_variants)
    not_supported_variant_analysis_summary["Fraction of all CIViC Variant Items"].append(fraction_not_supported_category_variant_of_civic)
    not_supported_variant_analysis_summary["Percent of all CIViC Variant Items"].append(percent_not_supported_category_variant_of_civic)
    not_supported_variant_analysis_summary["Fraction of Not Supported Variant Items"].append(fraction_not_supported_category_variant_of_total_not_supported)
    not_supported_variant_analysis_summary["Percent of Not Supported Variant Items"].append(percent_not_supported_category_variant_of_total_not_supported)
    not_supported_variant_analysis_summary["Fraction of Accepted Variant Items"].append(fraction_accepted_not_supported_category_variants)
    not_supported_variant_analysis_summary["Percent of Accepted Variant Items"].append(percentage_accepted_not_supported_category_variants)
    not_supported_variant_analysis_summary["Fraction of Not Accepted Variant Items"].append(fraction_not_accepted_not_supported_category_variants)
    not_supported_variant_analysis_summary["Percent of Not Accepted Variant Items"].append(percentage_not_accepted_not_supported_category_variants)

## Transform df for evidence analysis

In [47]:
not_supported_variants_add_evidence_df = transform_df_evidence_ids(not_supported_queries_df)
not_supported_variants_add_evidence_df.head()

,variant_id,gene_name,variant_name,category,variant_accepted,evidence_id
0,4170,VHL,NaN,Transcript Variant,False,10647
1,4417,ALK,FBXO11::ALK,Fusion,False,7428
2,4214,VHL,NaN,Transcript Variant,False,10752
3,4216,VHL,NaN,Transcript Variant,False,10754
4,4278,VHL,NaN,Transcript Variant,False,10958


In [48]:
not_supported_variants_add_evidence_df.loc[not_supported_variants_add_evidence_df['evidence_id'] == '']

,variant_id,gene_name,variant_name,category,variant_accepted,evidence_id
62,1027,ABL1,BCR::ABL D276G,Fusion,True,
118,4508,ABL1,BCR::ABL T315I,Fusion,False,
126,1025,ABL1,BCR::ABL Y253H,Fusion,True,
268,4270,EGFR,EGFR ITD,Rearrangements,False,
285,4020,EML4,EML4::NTRK3,Fusion,False,
503,3149,RELA,FUSION,Fusion,False,
715,3225,NTRK1,LMNA::NTRK1 e11-e10,Fusion,True,
716,3223,NTRK1,LMNA::NTRK1 e2-e10,Fusion,True,
717,3224,NTRK1,LMNA::NTRK1 e2-e11,Fusion,True,


In [49]:
not_supported_variants_add_evidence_df = transform_df_evidence(not_supported_variants_add_evidence_df)
not_supported_variants_add_evidence_df.head()

,variant_id,gene_name,variant_name,category,variant_accepted,evidence_id,evidence_status,evidence_rating,evidence_level
0,4170,VHL,NaN,Transcript Variant,False,10647,submitted,[2],[C]
1,4417,ALK,FBXO11::ALK,Fusion,False,7428,submitted,[3],[C]
2,4214,VHL,NaN,Transcript Variant,False,10752,submitted,[3],[C]
3,4216,VHL,NaN,Transcript Variant,False,10754,submitted,[3],[C]
4,4278,VHL,NaN,Transcript Variant,False,10958,submitted,[3],[C]


## Evidence analysis

In [50]:
not_supported_variants_add_evidence_df = evidence_analysis(not_supported_variants_add_evidence_df, VariantNormType.NOT_SUPPORTED)
not_supported_variants_add_evidence_df.head()

Number of Not Supported Variant Evidence items in CIViC: 4369 / 9914
Percentage of Not Supported Variant Evidence items in CIViC: 44.07%

Number of accepted Not Supported Variant Evidence items: 2215 / 4369
Percentage of accepted Not Supported Variant Evidence items: 50.70%

Number of submitted Not Supported Variant Evidence items: 2154 / 4369
Percentage of not submitted Not Supported Variant Evidence items: 49.30%


,variant_id,gene_name,variant_name,category,variant_accepted,evidence_id,evidence_status,evidence_rating,evidence_level,evidence_accepted
0,4170,VHL,NaN,Transcript Variant,False,10647,submitted,[2],[C],False
1,4417,ALK,FBXO11::ALK,Fusion,False,7428,submitted,[3],[C],False
2,4214,VHL,NaN,Transcript Variant,False,10752,submitted,[3],[C],False
3,4216,VHL,NaN,Transcript Variant,False,10754,submitted,[3],[C],False
4,4278,VHL,NaN,Transcript Variant,False,10958,submitted,[3],[C],False


### Not Supported Variant Evidence Analysis by Subcategory 

 List all the possible variant categories, have to use non unique file since evidence items are used more than once across groups


In [51]:
not_supported_variant_categories = not_supported_variants_add_evidence_df.category.unique()
[v for v in not_supported_variant_categories]

['Transcript Variant',
 'Fusion',
 'Region Defined Variant',
 'Protein Consequence',
 'Other',
 'Rearrangements',
 'Copy Number',
 'Expression',
 'Gene Function',
 'Genotypes Compound',
 'Epigenetic Modification',
 'Genotypes Easy']

Evidence ID's can qualify for more than one kind of variant category

In [52]:
duplicate = not_supported_variants_add_evidence_df[not_supported_variants_add_evidence_df.duplicated("evidence_id", keep=False)]
duplicate

,variant_id,gene_name,variant_name,category,variant_accepted,evidence_id,evidence_status,evidence_rating,evidence_level,evidence_accepted


In [379]:
df = not_supported_variants_add_evidence_df.copy()
df["evidence_score"] = ''
df

,variant_id,gene_name,variant_name,category,variant_accepted,evidence_id,evidence_status,evidence_rating,evidence_level,evidence_accepted,evidence_score
0,4170,VHL,NaN,Transcript Variant,False,10647,submitted,[2],[C],False,
1,4417,ALK,FBXO11::ALK,Fusion,False,7428,submitted,[3],[C],False,
2,4214,VHL,NaN,Transcript Variant,False,10752,submitted,[3],[C],False,
3,4216,VHL,NaN,Transcript Variant,False,10754,submitted,[3],[C],False,
4,4278,VHL,NaN,Transcript Variant,False,10958,submitted,[3],[C],False,
...,...,...,...,...,...,...,...,...,...,...,...
1619,3478,ESR2,underexpression beta-1,Other,False,9618,submitted,[4],[B],False,
1619,3478,ESR2,underexpression beta-1,Other,False,9619,submitted,[4],[B],False,
1620,3508,CD274,v242,Protein Consequence,False,9695,submitted,[4],[E],False,
1621,2422,NTRK3,~DEPRECATED-ETV6-NTRK3,Other,False,10692,submitted,[3],[C],False,


In [381]:
df_na = df[(df[("evidence_accepted")]!=False) & df["evidence_accepted"].isna()].copy()
df_na.to_csv('../civic_variants_lacking_any_evidence_df.csv', index=True)

In [301]:
df = df[(df[("evidence_accepted")]!=False) & df["evidence_accepted"].notna()].copy()
df['variant_accepted'].value_counts()

variant_accepted
True     2208
False       7
Name: count, dtype: int64

In [302]:
df.count()

variant_id           2215
gene_name            2215
variant_name         2161
category             2215
variant_accepted     2215
evidence_id          2215
evidence_status      2215
evidence_rating      2215
evidence_level       2215
evidence_accepted    2215
evidence_score       2215
dtype: int64

In [304]:
evidence_level_to_impact = {'A':10,
                            'B':5,
                            'C':3,
                            'D':1,
                            'E':0.5}

In [306]:
df['evidence_level'] = df["evidence_level"].apply(lambda x: x[0])

In [307]:
df["evidence_score"] = df["evidence_level"].map(evidence_level_to_impact)
df

,variant_id,gene_name,variant_name,category,variant_accepted,evidence_id,evidence_status,evidence_rating,evidence_level,evidence_accepted,evidence_score
7,2930,VHL,NaN,Transcript Variant,True,7892,accepted,[3],C,True,3.0
9,785,CHEK2,1100DELC,Protein Consequence,True,1850,accepted,[3],B,True,5.0
13,823,EPCAM,3' Exon Deletion,Region Defined Variant,True,1901,accepted,[4],B,True,5.0
15,433,HIF1A,3' UTR Polymorphism,Region Defined Variant,True,1031,accepted,[2],B,True,5.0
17,2367,VHL,3p26.3-25.3 11Mb del,Region Defined Variant,True,6287,accepted,[3],C,True,3.0
...,...,...,...,...,...,...,...,...,...,...,...
1601,272,CDKN2A,p16 Expression,Expression,True,1314,accepted,[2],B,True,5.0
1603,3313,CDKN1A,rs1059234,Other,True,9244,accepted,[3],B,True,5.0
1605,256,KIT,rs17084733,Other,True,666,accepted,[3],B,True,5.0
1606,2671,CDKN1A,rs1801270,Other,True,7227,accepted,[3],B,True,5.0


In [308]:
df.sort_values(by=["variant_id"])

,variant_id,gene_name,variant_name,category,variant_accepted,evidence_id,evidence_status,evidence_rating,evidence_level,evidence_accepted,evidence_score
58,1,ABL1,BCR::ABL,Fusion,True,6244,accepted,[2],D,True,1.0
58,1,ABL1,BCR::ABL,Fusion,True,7039,accepted,[1],C,True,3.0
58,1,ABL1,BCR::ABL,Fusion,True,233,accepted,[4],D,True,1.0
58,1,ABL1,BCR::ABL,Fusion,True,2659,accepted,[3],D,True,1.0
58,1,ABL1,BCR::ABL,Fusion,True,2661,accepted,[3],D,True,1.0
...,...,...,...,...,...,...,...,...,...,...,...
891,4440,BARD1,Mutation,Region Defined Variant,True,11209,accepted,[2],A,True,10.0
892,4441,CHEK1,Mutation,Region Defined Variant,True,11210,accepted,[2],A,True,10.0
894,4443,RAD51B,Mutation,Region Defined Variant,True,11212,accepted,[2],A,True,10.0
896,4445,RAD51D,Mutation,Region Defined Variant,True,11214,accepted,[2],A,True,10.0


In [309]:
df_var = df.groupby('variant_id').aggregate({'gene_name':'first',
                                             'variant_name':'first',
                                             'category':'first',
                                             'evidence_id':'count',
                                             'evidence_score':'sum'})
df_var = df_var.rename(columns={'evidence_id':'#_evidence_items',
                                'evidence_score':'evidence_score_sum'})
df_var

,gene_name,variant_name,category,#_evidence_items,evidence_score_sum
variant_id,,,,,
1,ABL1,BCR::ABL,Fusion,129,320.0
5,ALK,EML4::ALK,Fusion,47,91.0
17,BRAF,V600,Protein Consequence,22,123.0
19,CCND1,Expression,Expression,2,10.0
20,CCND1,Overexpression,Expression,8,36.0
...,...,...,...,...,...
4440,BARD1,Mutation,Region Defined Variant,1,10.0
4441,CHEK1,Mutation,Region Defined Variant,1,10.0
4443,RAD51B,Mutation,Region Defined Variant,1,10.0


In [310]:
df_cat = df_var.groupby('category').aggregate({'gene_name':'count',
                                               '#_evidence_items':'sum',
                                               'evidence_score_sum':'sum'})
df_cat = df_cat.rename(columns={'evidence_score_sum':'impact',
                                'gene_name':'number_of_variants'})
df_cat['average_impact_per_variant'] = df_cat['impact']/df_cat['number_of_variants']
df_cat.sort_values(by=["impact"], ascending=False)

,number_of_variants,#_evidence_items,impact,average_impact_per_variant
category,,,,
Region Defined Variant,135,558,2542.0,18.829630
Fusion,199,737,2270.5,11.409548
Expression,180,342,1235.0,6.861111
Protein Consequence,70,191,837.5,11.964286
Gene Function,49,154,540.5,11.030612
Other,43,60,234.0,5.441860
Rearrangements,12,53,204.5,17.041667
Transcript Variant,51,54,156.0,3.058824
Copy Number,18,27,83.0,4.611111


In [311]:
df_cat.sum()

number_of_variants             783.000000
#_evidence_items              2215.000000
impact                        8271.000000
average_impact_per_variant     110.355791
dtype: float64

In [312]:
df_both_evid = not_supported_variants_add_evidence_df
df_both_evid["evidence_score"] = ''
df_both_evid = df_both_evid[df_both_evid['evidence_accepted'].notna()].copy()
df_both_evid

,variant_id,gene_name,variant_name,category,variant_accepted,evidence_id,evidence_status,evidence_rating,evidence_level,evidence_accepted,evidence_score
0,4170,VHL,NaN,Transcript Variant,False,10647,submitted,[2],[C],False,
1,4417,ALK,FBXO11::ALK,Fusion,False,7428,submitted,[3],[C],False,
2,4214,VHL,NaN,Transcript Variant,False,10752,submitted,[3],[C],False,
3,4216,VHL,NaN,Transcript Variant,False,10754,submitted,[3],[C],False,
4,4278,VHL,NaN,Transcript Variant,False,10958,submitted,[3],[C],False,
...,...,...,...,...,...,...,...,...,...,...,...
1619,3478,ESR2,underexpression beta-1,Other,False,9618,submitted,[4],[B],False,
1619,3478,ESR2,underexpression beta-1,Other,False,9619,submitted,[4],[B],False,
1620,3508,CD274,v242,Protein Consequence,False,9695,submitted,[4],[E],False,
1621,2422,NTRK3,~DEPRECATED-ETV6-NTRK3,Other,False,10692,submitted,[3],[C],False,


In [313]:
df_both_evid['evidence_level'] = df_both_evid["evidence_level"].apply(lambda y: y[0])

In [314]:
df_both_evid["evidence_score"] = df_both_evid["evidence_level"].map(evidence_level_to_impact)
df_both_evid

,variant_id,gene_name,variant_name,category,variant_accepted,evidence_id,evidence_status,evidence_rating,evidence_level,evidence_accepted,evidence_score
0,4170,VHL,NaN,Transcript Variant,False,10647,submitted,[2],C,False,3.0
1,4417,ALK,FBXO11::ALK,Fusion,False,7428,submitted,[3],C,False,3.0
2,4214,VHL,NaN,Transcript Variant,False,10752,submitted,[3],C,False,3.0
3,4216,VHL,NaN,Transcript Variant,False,10754,submitted,[3],C,False,3.0
4,4278,VHL,NaN,Transcript Variant,False,10958,submitted,[3],C,False,3.0
...,...,...,...,...,...,...,...,...,...,...,...
1619,3478,ESR2,underexpression beta-1,Other,False,9618,submitted,[4],B,False,5.0
1619,3478,ESR2,underexpression beta-1,Other,False,9619,submitted,[4],B,False,5.0
1620,3508,CD274,v242,Protein Consequence,False,9695,submitted,[4],E,False,0.5
1621,2422,NTRK3,~DEPRECATED-ETV6-NTRK3,Other,False,10692,submitted,[3],C,False,3.0


In [315]:
df_both_evid.sort_values(by=["variant_id"])
df_both_evid_var = df_both_evid.groupby('variant_id').aggregate({'gene_name':'first',
                                             'variant_name':'first',
                                             'category':'first',
                                             'evidence_id':'count',
                                             'evidence_score':'sum'})
df_both_evid_var = df_both_evid_var.rename(columns={'evidence_id':'#_evidence_items',
                                'evidence_score':'evidence_score_sum'})
df_both_evid_var

,gene_name,variant_name,category,#_evidence_items,evidence_score_sum
variant_id,,,,,
1,ABL1,BCR::ABL,Fusion,185,494.0
5,ALK,EML4::ALK,Fusion,95,167.0
17,BRAF,V600,Protein Consequence,26,153.0
19,CCND1,Expression,Expression,2,10.0
20,CCND1,Overexpression,Expression,10,40.0
...,...,...,...,...,...
4491,ERBB2,Exon 17 V659E,Region Defined Variant,1,5.0
4492,ERBB2,Exon 8 S310F,Region Defined Variant,2,2.0
4495,ALK,DCTN1::ALK,Fusion,1,3.0


In [316]:
df_both_evid_cat = df_both_evid_var.groupby('category').aggregate({'gene_name':'count',
                                               '#_evidence_items':'sum',
                                               'evidence_score_sum':'sum'})
df_both_evid_cat = df_both_evid_cat.rename(columns={'evidence_score_sum':'impact',
                                'gene_name':'number_of_variants'})
df_both_evid_cat['average_impact_per_variant'] = df_both_evid_cat['impact']/df_both_evid_cat['number_of_variants']
df_both_evid_cat.sort_values(by=["impact"], ascending=False)

,number_of_variants,#_evidence_items,impact,average_impact_per_variant
category,,,,
Region Defined Variant,190,955,4125.5,21.713158
Fusion,283,1198,3741.5,13.220848
Expression,286,597,2029.5,7.096154
Transcript Variant,360,440,1318.0,3.661111
Protein Consequence,126,292,1174.0,9.317460
Gene Function,85,335,1083.0,12.741176
Other,85,154,565.5,6.652941
Rearrangements,46,129,415.5,9.032609
Copy Number,32,61,187.0,5.843750


In [317]:
(df_both_evid_cat['impact'] - df_cat['impact']).sort_values(ascending=False)

category
Region Defined Variant     1583.5
Fusion                     1471.0
Transcript Variant         1162.0
Expression                  794.5
Gene Function               542.5
Protein Consequence         336.5
Other                       331.5
Rearrangements              211.0
Copy Number                 104.0
Genotypes Easy               44.0
Genotypes Compound           11.0
Epigenetic Modification      10.0
Name: impact, dtype: float64

In [318]:
df_both_evid_cat.to_csv('../civic_both_evidence_cat_impact_df.csv', index=True)
df_cat.to_csv('../civic_accepted_evidence_only_impact_df.csv', index=True)


In [319]:
not_supported_variant_evidence_summary = {
    "Category": VARIANT_CATEGORY_VALUES,
    "Count of CIViC Evidence Items per Category":[],
    "Fraction of CIViC Evidence Items": [],
    "Percent of all CIViC Evidence Items": [],
    "Fraction of Not Supported Variant Evidence Items": [],
    "Percent of Not Supported Variant Evidence Items": [],
    "Fraction of Accepted Evidence Items": [],
    "Percent of Accepted Evidence Items": [],
    "Fraction of Submitted Evidence Items": [],
    "Percent of Submitted Evidence Items": []
}

In [320]:
not_supported_variant_categories_evidence_summary_data = dict()
total_number_not_supported_variant_unique_evidence_items = len(set(not_supported_variants_add_evidence_df.evidence_id))

for category in VARIANT_CATEGORY_VALUES:
    not_supported_variant_categories_evidence_summary_data[category] = {}
    evidence_category_df = not_supported_variants_add_evidence_df[not_supported_variants_add_evidence_df.category == category]
    evidence_category_df = evidence_category_df.drop_duplicates(subset=["evidence_id"])

    # Count
    number_unique_not_supported_category_evidence = len(set(evidence_category_df.evidence_id))
    not_supported_variant_categories_evidence_summary_data[category]["number_unique_not_supported_category_evidence"] = number_unique_not_supported_category_evidence

    # Fraction
    fraction_not_supported_category_variant_evidence_of_civic = f"{number_unique_not_supported_category_evidence} / {total_ac_sub_evidence}"
    not_supported_variant_categories_evidence_summary_data[category]["fraction_not_supported_category_variant_evidence_of_civic"] = fraction_not_supported_category_variant_evidence_of_civic

    # Percent
    percent_not_supported_category_variant_evidence_of_civic = f"{number_unique_not_supported_category_evidence / total_ac_sub_evidence * 100:.2f}%"
    not_supported_variant_categories_evidence_summary_data[category]["percent_not_supported_category_variant_evidence_of_civic"] = percent_not_supported_category_variant_evidence_of_civic

    # Not supported fraction
    fraction_not_supported_category_variant_evidence_of_total_not_supported = f"{number_unique_not_supported_category_evidence} / {total_number_not_supported_variant_unique_evidence_items}"
    not_supported_variant_categories_evidence_summary_data[category]["fraction_not_supported_category_variant_evidence_of_total_not_supported"] = fraction_not_supported_category_variant_evidence_of_total_not_supported

    # Not supported percent
    percent_not_supported_category_variant_evidence_of_total_not_supported = f"{number_unique_not_supported_category_evidence / total_number_not_supported_variant_unique_evidence_items * 100:.2f}%"
    not_supported_variant_categories_evidence_summary_data[category]["percent_not_supported_category_variant_evidence_of_total_not_supported"] = percent_not_supported_category_variant_evidence_of_total_not_supported

    # Accepted fraction
    number_accepted_not_supported_category_variant_evidence = evidence_category_df.evidence_accepted.sum()
    fraction_accepted_evidence_not_supported_category_variants = f"{number_accepted_not_supported_category_variant_evidence} / {number_unique_not_supported_category_evidence}"
    not_supported_variant_categories_evidence_summary_data[category]["fraction_accepted_evidence_not_supported_category_variants"] = fraction_accepted_evidence_not_supported_category_variants

    # Accepted percent
    percentage_accepted_evidence_not_supported_category_variants = f"{number_accepted_not_supported_category_variant_evidence / number_unique_not_supported_category_evidence * 100:.2f}%"
    not_supported_variant_categories_evidence_summary_data[category]["percentage_accepted_evidence_not_supported_category_variants"] = percentage_accepted_evidence_not_supported_category_variants

    # Submitted fraction
    number_submitted_not_supported_category_variant_evidence = number_unique_not_supported_category_evidence - evidence_category_df.evidence_accepted.sum()
    fraction_submitted_evidence_not_supported_category_variants = f"{number_submitted_not_supported_category_variant_evidence} / {number_unique_not_supported_category_evidence}"
    not_supported_variant_categories_evidence_summary_data[category]["fraction_submitted_evidence_not_supported_category_variants"] = fraction_submitted_evidence_not_supported_category_variants

    # Submitted percent
    percentage_submitted_evidence_not_supported_category_variants = f"{number_submitted_not_supported_category_variant_evidence / number_unique_not_supported_category_evidence * 100:.2f}%"
    not_supported_variant_categories_evidence_summary_data[category]["percentage_submitted_evidence_not_supported_category_variants"] = percentage_submitted_evidence_not_supported_category_variants
   
    not_supported_variant_evidence_summary["Count of CIViC Evidence Items per Category"].append(number_unique_not_supported_category_evidence)
    not_supported_variant_evidence_summary["Fraction of CIViC Evidence Items"].append(fraction_not_supported_category_variant_evidence_of_civic)
    not_supported_variant_evidence_summary["Percent of all CIViC Evidence Items"].append(percent_not_supported_category_variant_evidence_of_civic)
    not_supported_variant_evidence_summary["Fraction of Not Supported Variant Evidence Items"].append(fraction_not_supported_category_variant_evidence_of_total_not_supported)
    not_supported_variant_evidence_summary["Percent of Not Supported Variant Evidence Items"].append(percent_not_supported_category_variant_evidence_of_total_not_supported)
    not_supported_variant_evidence_summary["Fraction of Accepted Evidence Items"].append(fraction_accepted_evidence_not_supported_category_variants)
    not_supported_variant_evidence_summary["Percent of Accepted Evidence Items"].append(percentage_accepted_evidence_not_supported_category_variants)
    not_supported_variant_evidence_summary["Fraction of Submitted Evidence Items"].append(fraction_submitted_evidence_not_supported_category_variants)
    not_supported_variant_evidence_summary["Percent of Submitted Evidence Items"].append(percentage_submitted_evidence_not_supported_category_variants)

## Impact
molecular profile score

### Import molecular profile id

In [321]:
not_supported_variants_add_molprof_df = transform_df_mp_id(not_supported_queries_df)
not_supported_variants_add_molprof_df.head()

,variant_id,gene_name,variant_name,category,variant_accepted,molecular_profile_id
0,4170,VHL,NaN,Transcript Variant,False,[4038]
1,4417,ALK,FBXO11::ALK,Fusion,False,[4350]
2,4214,VHL,NaN,Transcript Variant,False,[4082]
3,4216,VHL,NaN,Transcript Variant,False,[4084]
4,4278,VHL,NaN,Transcript Variant,False,[4146]


### Import molecular profile scores

In [322]:
not_supported_variants_add_molprof_df = transform_df_mp_score(not_supported_variants_add_molprof_df)
not_supported_variants_add_molprof_df.head()

,variant_id,gene_name,variant_name,category,variant_accepted,molecular_profile_id,molecular_profile_score
0,4170,VHL,NaN,Transcript Variant,False,[4038],[0.0]
1,4417,ALK,FBXO11::ALK,Fusion,False,[4350],[0.0]
2,4214,VHL,NaN,Transcript Variant,False,[4082],[0.0]
3,4216,VHL,NaN,Transcript Variant,False,[4084],[0.0]
4,4278,VHL,NaN,Transcript Variant,False,[4146],[0.0]


In [323]:
not_supported_variants_add_molprof_df = transform_df_mp_score_sum(not_supported_variants_add_molprof_df)
not_supported_variants_add_molprof_df.head()

,variant_id,gene_name,variant_name,category,variant_accepted,molecular_profile_id,molecular_profile_score,molecular_profile_score_sum
0,4170,VHL,NaN,Transcript Variant,False,[4038],[0.0],0.0
1,4417,ALK,FBXO11::ALK,Fusion,False,[4350],[0.0],0.0
2,4214,VHL,NaN,Transcript Variant,False,[4082],[0.0],0.0
3,4216,VHL,NaN,Transcript Variant,False,[4084],[0.0],0.0
4,4278,VHL,NaN,Transcript Variant,False,[4146],[0.0],0.0


In [324]:
not_supported_variants_add_molprof_df[(not_supported_variants_add_molprof_df["molecular_profile_score_sum"]==0.0)& (not_supported_variants_add_molprof_df["variant_accepted"]==True)]

,variant_id,gene_name,variant_name,category,variant_accepted,molecular_profile_id,molecular_profile_score,molecular_profile_score_sum
62,1027,ABL1,BCR::ABL D276G,Fusion,True,,,0.0
126,1025,ABL1,BCR::ABL Y253H,Fusion,True,,,0.0
715,3225,NTRK1,LMNA::NTRK1 e11-e10,Fusion,True,,,0.0
716,3223,NTRK1,LMNA::NTRK1 e2-e10,Fusion,True,,,0.0
717,3224,NTRK1,LMNA::NTRK1 e2-e11,Fusion,True,,,0.0
776,1247,BRCA2,M1R,Other,True,[1221],[0.0],0.0


In [325]:
not_supported_variants_add_molprof_df["molecular_profile_score_sum"].max()

862.5

In [326]:
not_supported_variants_add_molprof_df[(not_supported_variants_add_molprof_df["molecular_profile_score_sum"]!=0.0)]

,variant_id,gene_name,variant_name,category,variant_accepted,molecular_profile_id,molecular_profile_score,molecular_profile_score_sum
7,2930,VHL,NaN,Transcript Variant,True,[2799],[7.5],7.5
9,785,CHEK2,1100DELC,Protein Consequence,True,[766],[15.0],15.0
13,823,EPCAM,3' Exon Deletion,Region Defined Variant,True,[801],[20.0],20.0
15,433,HIF1A,3' UTR Polymorphism,Region Defined Variant,True,[429],[10.0],10.0
17,2367,VHL,3p26.3-25.3 11Mb del,Region Defined Variant,True,[2240],[7.5],7.5
...,...,...,...,...,...,...,...,...
1601,272,CDKN2A,p16 Expression,Expression,True,[268],[180.0],180.0
1603,3313,CDKN1A,rs1059234,Other,True,[3181],[15.0],15.0
1605,256,KIT,rs17084733,Other,True,[252],[15.0],15.0
1606,2671,CDKN1A,rs1801270,Other,True,[2540],[15.0],15.0


### Impact by Subcategory

In [327]:
not_supported_impact_summary = {
    "Category": VARIANT_CATEGORY_VALUES,
    "CIVIC Total Sum Impact Score": [],
    'Average Impact Score per Variant': [],
    'Average Impact Score per Evidence Item':[],
    "Total Number Evidence Items": [v["number_unique_not_supported_category_evidence"] for v in not_supported_variant_categories_evidence_summary_data.values()],
    "% Accepted Evidence Items": [v["percentage_accepted_evidence_not_supported_category_variants"] for v in not_supported_variant_categories_evidence_summary_data.values()],
    "Total Number Variants": [v["number_unique_not_supported_category_variants"] for v in not_supported_variant_categories_summary_data.values()]
}

In [328]:
not_supported_variant_categories_impact_data = dict()
for category in VARIANT_CATEGORY_VALUES:
    not_supported_variant_categories_impact_data[category] = {}
    impact_category_df = not_supported_variants_add_molprof_df[not_supported_variants_add_molprof_df.category == category]

    total_sum_not_supported_category_impact = impact_category_df["molecular_profile_score_sum"].sum()
    not_supported_variant_categories_impact_data[category]["total_sum_not_supported_category_impact"] = total_sum_not_supported_category_impact

    avg_impact_score_variant = total_sum_not_supported_category_impact/ number_unique_not_supported_category_variants
    not_supported_variant_categories_impact_data[category]["avg_impact_score_variant"]=  avg_impact_score_variant

    avg_impact_score_evidence = total_sum_not_supported_category_impact/ number_unique_not_supported_category_evidence
    not_supported_variant_categories_impact_data[category]["avg_impact_score_evidence"]=  avg_impact_score_evidence

    not_supported_impact_summary["CIVIC Total Sum Impact Score"].append(total_sum_not_supported_category_impact)
    not_supported_impact_summary["Average Impact Score per Variant"].append(avg_impact_score_variant)
    not_supported_impact_summary["Average Impact Score per Evidence Item"].append(avg_impact_score_evidence)


    print(number_unique_not_supported_category_variants)
    print(f"{category}: {total_sum_not_supported_category_impact}")

385
Expression: 3618.0
385
Epigenetic Modification: 285.5
385
Fusion: 6600.75
385
Protein Consequence: 2683.75
385
Gene Function: 1805.5
385
Rearrangements: 653.0
385
Copy Number: 225.0
385
Other: 673.5
385
Genotypes Easy: 195.0
385
Genotypes Compound: 117.5
385
Region Defined Variant: 8353.5
385
Transcript Variant: 356.5


In [329]:
not_supported_variant_impact_df = pd.DataFrame(not_supported_impact_summary)

In [330]:
not_supported_variant_impact_df

,Category,CIVIC Total Sum Impact Score,Average Impact Score per Variant,Average Impact Score per Evidence Item,Total Number Evidence Items,% Accepted Evidence Items,Total Number Variants
0,Expression,3618.00,9.397403,7.681529,618,55.34%,291
1,Epigenetic Modification,285.50,0.741558,0.606157,24,91.67%,15
2,Fusion,6600.75,17.144805,14.014331,1239,59.48%,308
3,Protein Consequence,2683.75,6.970779,5.697983,301,63.46%,128
4,Gene Function,1805.50,4.689610,3.833333,348,44.25%,90
5,Rearrangements,653.00,1.696104,1.386412,136,38.97%,52
6,Copy Number,225.00,0.584416,0.477707,64,42.19%,34
7,Other,673.50,1.749351,1.429936,162,37.04%,93
8,Genotypes Easy,195.00,0.506494,0.414013,23,56.52%,11
9,Genotypes Compound,117.50,0.305195,0.249469,7,57.14%,6


In [331]:
not_supported_variant_impact_df.to_csv('../not_supported_variant_impact_df.csv', index=False)

# Summary

## Variant Analysis

In [332]:
all_variant_df = pd.DataFrame(variant_analysis_summary)

In [333]:
all_variant_df["Percentage of all CIViC Variant Items"] = all_variant_df["Fraction of all CIViC Variant Items"].astype(str) + "  (" + all_variant_df["Percentage of all CIViC Variant Items"] + ")"
all_variant_df["Percentage of Accepted Variant Items"] = all_variant_df["Fraction of Accepted Variant Items"].astype(str) + "  (" + all_variant_df["Percentage of Accepted Variant Items"] + ")"
all_variant_df["Percentage of Not Accepted Variant Items"] = all_variant_df["Fraction of Not Accepted Variant Items"].astype(str) + "  (" + all_variant_df["Percentage of Not Accepted Variant Items"] + ")"

In [334]:
all_variant_df = all_variant_df.drop(["Fraction of all CIViC Variant Items", "Fraction of Accepted Variant Items", "Fraction of Not Accepted Variant Items"], axis=1)

In [335]:
all_variant_percent_status_df = all_variant_df.drop(["Percentage of all CIViC Variant Items", "Count of CIViC Variant Items per Category"], axis=1)

for_merge_all_variant_percent_of_civic_df = all_variant_df.drop(["Percentage of Accepted Variant Items", "Percentage of Not Accepted Variant Items"], axis=1)

all_variant_percent_of_civic_df = for_merge_all_variant_percent_of_civic_df.drop(["Count of CIViC Variant Items per Category"], axis=1)

In [336]:
for_merge_all_variant_percent_of_civic_df.to_csv('../for_merge_all_variant_percent_of_civic_df.csv', index=False)


Summary Table 1: The table below shows the 3 categories that CIViC variants were divided into after normalization and what percentage they make up of all variants in CIViC data. 

In [337]:
all_variant_percent_of_civic_df = all_variant_percent_of_civic_df.set_index("Variant Category")
all_variant_percent_of_civic_df

,Percentage of all CIViC Variant Items
Variant Category,
Normalized,1869 / 3581 (52.19%)
Unable to Normalize,63 / 3581 (1.76%)
Not Supported,1622 / 3581 (45.29%)


In [338]:
civic_summary_table_1 = all_variant_percent_of_civic_df

Summary Table 2: The table below shows the 3 categories that CIViC variants were divided into after normalization and what percentage of the variants in each category are accepted (have at least one evidence item that is accepted) or not.

In [339]:
all_variant_percent_status_df = all_variant_percent_status_df.set_index("Variant Category")
all_variant_percent_status_df

,Percentage of Accepted Variant Items,Percentage of Not Accepted Variant Items
Variant Category,,
Normalized,866 / 1869 (46.33%),1003 / 1869 (53.67%)
Unable to Normalize,9 / 63 (14.29%),54 / 63 (85.71%)
Not Supported,788 / 1622 (48.58%),834 / 1622 (51.42%)


In [340]:
civic_summary_table_2 = all_variant_percent_status_df

In [341]:
not_supported_variant_df = pd.DataFrame(not_supported_variant_analysis_summary)

In [342]:
not_supported_variant_df["Percent of Not Supported Variant Items"] = not_supported_variant_df["Fraction of Not Supported Variant Items"].astype(str) + "  (" + not_supported_variant_df["Percent of Not Supported Variant Items"] + ")"
not_supported_variant_df["Percent of all CIViC Variant Items"] = not_supported_variant_df["Fraction of all CIViC Variant Items"].astype(str) + "  (" + not_supported_variant_df["Percent of all CIViC Variant Items"] + ")"
not_supported_variant_df["Percent of Accepted Variant Items"] = not_supported_variant_df["Fraction of Accepted Variant Items"].astype(str) + "  (" + not_supported_variant_df["Percent of Accepted Variant Items"] + ")"
not_supported_variant_df["Percent of Not Accepted Variant Items"] = not_supported_variant_df["Fraction of Not Accepted Variant Items"].astype(str) + "  (" + not_supported_variant_df["Percent of Not Accepted Variant Items"] + ")"

In [343]:
not_supported_variant_df = not_supported_variant_df.drop(["Fraction of Not Supported Variant Items", "Fraction of all CIViC Variant Items", "Fraction of Accepted Variant Items", "Fraction of Not Accepted Variant Items"], axis=1)

In [344]:
for_merge_not_supported_variant_percent_of_civic_df = not_supported_variant_df.drop(["Percent of Not Supported Variant Items", "Percent of Accepted Variant Items", "Percent of Not Accepted Variant Items"], axis=1)

not_supported_variant_percent_of_civic_df = for_merge_not_supported_variant_percent_of_civic_df.drop(["Count of CIViC Variant Items per Category"], axis=1)

not_supported_variant_percent_of_not_supported_df = not_supported_variant_df.drop(["Percent of all CIViC Variant Items", "Count of CIViC Variant Items per Category", "Percent of Accepted Variant Items", "Percent of Not Accepted Variant Items"], axis=1)

not_supported_variant_percent_evidence_df = not_supported_variant_df.drop(["Percent of all CIViC Variant Items", "Percent of Not Supported Variant Items","Count of CIViC Variant Items per Category"], axis=1)

In [345]:
for_merge_not_supported_variant_percent_of_civic_df.to_csv('../for_merge_not_supported_variant_percent_of_civic_df.csv', index=False)

Summary Table 3: The table below shows the categories that the Not Supported variants were broken into and what percentage of all CIViC variants they make up. These percentages will not add up to 100% because Not Supported variants make up 45.62% of all CIViC variants. 

In [346]:
not_supported_variant_percent_of_civic_df = not_supported_variant_percent_of_civic_df.set_index("Category")
not_supported_variant_percent_of_civic_df

,Percent of all CIViC Variant Items
Category,
Expression,291 / 3581 (8.13%)
Epigenetic Modification,15 / 3581 (0.42%)
Fusion,308 / 3581 (8.60%)
Protein Consequence,128 / 3581 (3.57%)
Gene Function,90 / 3581 (2.51%)
Rearrangements,52 / 3581 (1.45%)
Copy Number,34 / 3581 (0.95%)
Other,93 / 3581 (2.60%)
Genotypes Easy,11 / 3581 (0.31%)


In [347]:
civic_summary_table_3 = not_supported_variant_percent_of_civic_df

Summary Table 4: The table below shows the Not Supported variants broken up into 12 sub categories and what percent each sub category take up in Not Supported variant group.

In [348]:
not_supported_variant_percent_of_not_supported_df = not_supported_variant_percent_of_not_supported_df.set_index("Category")
not_supported_variant_percent_of_not_supported_df

,Percent of Not Supported Variant Items
Category,
Expression,291 / 1622 (17.94%)
Epigenetic Modification,15 / 1622 (0.92%)
Fusion,308 / 1622 (18.99%)
Protein Consequence,128 / 1622 (7.89%)
Gene Function,90 / 1622 (5.55%)
Rearrangements,52 / 1622 (3.21%)
Copy Number,34 / 1622 (2.10%)
Other,93 / 1622 (5.73%)
Genotypes Easy,11 / 1622 (0.68%)


In [349]:
civic_summary_table_4= not_supported_variant_percent_of_not_supported_df

Summary Table 5: The table below shows the percent of variant items in each Not Supported variant sub category that are accepted(have at least one evidence item that is accepted) or not, per category.

In [350]:
not_supported_variant_percent_evidence_df = not_supported_variant_percent_evidence_df.set_index("Category")
not_supported_variant_percent_evidence_df

,Percent of Accepted Variant Items,Percent of Not Accepted Variant Items
Category,,
Expression,180 / 291 (61.86%),111 / 291 (38.14%)
Epigenetic Modification,14 / 15 (93.33%),1 / 15 (6.67%)
Fusion,204 / 308 (66.23%),104 / 308 (33.77%)
Protein Consequence,70 / 128 (54.69%),58 / 128 (45.31%)
Gene Function,49 / 90 (54.44%),41 / 90 (45.56%)
Rearrangements,12 / 52 (23.08%),40 / 52 (76.92%)
Copy Number,19 / 34 (55.88%),15 / 34 (44.12%)
Other,43 / 93 (46.24%),50 / 93 (53.76%)
Genotypes Easy,8 / 11 (72.73%),3 / 11 (27.27%)


In [351]:
civic_summary_table_5 = not_supported_variant_percent_evidence_df

## Evidence Analysis

In [352]:
all_variant_evidence_df = pd.DataFrame(evidence_analysis_summary)

In [353]:
all_variant_evidence_df["Percentage of all CIViC Evidence Items"] = all_variant_evidence_df["Fraction of all CIViC Evidence Items"].astype(str) + "  (" + all_variant_evidence_df["Percentage of all CIViC Evidence Items"] + ")"
all_variant_evidence_df["Percentage of Accepted Evidence Items"] = all_variant_evidence_df["Fraction of Accepted Evidence Items"].astype(str) + "  (" + all_variant_evidence_df["Percentage of Accepted Evidence Items"] + ")"
all_variant_evidence_df["Percentage of Submitted Evidence Items"] = all_variant_evidence_df["Fraction of Submitted Evidence Items"].astype(str) + "  (" + all_variant_evidence_df["Percentage of Submitted Evidence Items"] + ")"

In [354]:
all_variant_evidence_df = all_variant_evidence_df.drop(["Fraction of all CIViC Evidence Items", "Fraction of Accepted Evidence Items", "Fraction of Submitted Evidence Items"], axis=1)

In [355]:
for_merge_all_variant_evidence_percent_of_civic_df = all_variant_evidence_df.drop(["Percentage of Accepted Evidence Items", "Percentage of Submitted Evidence Items"], axis=1)

all_variant_evidence_percent_of_civic_df = for_merge_all_variant_evidence_percent_of_civic_df.drop(["Count of CIViC Evidence Items per Category"], axis=1)

all_variant_evidence_percent_evidence_df = all_variant_evidence_df.drop(["Percentage of all CIViC Evidence Items", "Count of CIViC Evidence Items per Category"], axis=1)

In [356]:
for_merge_all_variant_evidence_percent_of_civic_df.to_csv('../for_merge_all_variant_evidence_percent_of_civic_df.csv', index=False)

Summary Table 6: The table below shows what percentage of all evidence items in CIViC are associated with Normalized, Unable to Normalize, and Not Supported variants. This will not add up to 100% because evidence itmes may be used across multiple variants.

In [357]:
all_variant_evidence_percent_of_civic_df = all_variant_evidence_percent_of_civic_df.set_index("Variant Category")
all_variant_evidence_percent_of_civic_df

,Percentage of all CIViC Evidence Items
Variant Category,
Normalized,5960 / 9914 (60.12%)
Unable to Normalize,80 / 9914 (0.81%)
Not Supported,4369 / 9914 (44.07%)


In [358]:
civic_summary_table_6 = all_variant_evidence_percent_of_civic_df

Summmary Table 7: The table below shows the percentage of accepted and sumbitted evidence items per category of variants. 

In [359]:
all_variant_evidence_percent_evidence_df = all_variant_evidence_percent_evidence_df.set_index("Variant Category")
all_variant_evidence_percent_evidence_df

,Percentage of Accepted Evidence Items,Percentage of Submitted Evidence Items
Variant Category,,
Normalized,2033 / 5960 (34.11%),3927 / 5960 (65.89%)
Unable to Normalize,14 / 80 (17.50%),66 / 80 (82.50%)
Not Supported,2215 / 4369 (50.70%),2154 / 4369 (49.30%)


In [360]:
civic_summary_table_7 = all_variant_evidence_percent_evidence_df

In [361]:
not_supported_variant_evidence_df = pd.DataFrame(not_supported_variant_evidence_summary)

In [362]:
not_supported_variant_evidence_df["Percent of all CIViC Evidence Items"] = not_supported_variant_evidence_df["Fraction of CIViC Evidence Items"].astype(str) + "  (" + not_supported_variant_evidence_df["Percent of all CIViC Evidence Items"] + ")"
not_supported_variant_evidence_df["Percent of Not Supported Variant Evidence Items"] = not_supported_variant_evidence_df["Fraction of Not Supported Variant Evidence Items"].astype(str) + "  (" + not_supported_variant_evidence_df["Percent of Not Supported Variant Evidence Items"] + ")"
not_supported_variant_evidence_df["Percent of Accepted Evidence Items"] = not_supported_variant_evidence_df["Fraction of Accepted Evidence Items"].astype(str) + "  (" + not_supported_variant_evidence_df["Percent of Accepted Evidence Items"] + ")"
not_supported_variant_evidence_df["Percent of Submitted Evidence Items"] = not_supported_variant_evidence_df["Fraction of Submitted Evidence Items"].astype(str) + "  (" + not_supported_variant_evidence_df["Percent of Submitted Evidence Items"] + ")"

In [363]:
not_supported_variant_evidence_df = not_supported_variant_evidence_df.drop(["Fraction of CIViC Evidence Items", "Fraction of Not Supported Variant Evidence Items", "Fraction of Accepted Evidence Items", "Fraction of Submitted Evidence Items"], axis=1)

In [364]:
for_merge_not_supported_variant_evidence_percent_of_civic_df = not_supported_variant_evidence_df.drop(["Percent of Not Supported Variant Evidence Items", "Percent of Accepted Evidence Items", "Percent of Submitted Evidence Items"], axis=1)

not_supported_variant_evidence_percent_of_civic_df = for_merge_not_supported_variant_evidence_percent_of_civic_df.drop(["Count of CIViC Evidence Items per Category"], axis=1)

not_supported_variant_evidence_percent_of_not_supported_df = not_supported_variant_evidence_df.drop(["Percent of all CIViC Evidence Items", "Percent of Accepted Evidence Items", "Percent of Submitted Evidence Items", "Count of CIViC Evidence Items per Category"], axis=1)

not_supported_variant_evidence_percent_evidence_df = not_supported_variant_evidence_df.drop(["Percent of Not Supported Variant Evidence Items", "Percent of all CIViC Evidence Items", "Count of CIViC Evidence Items per Category"], axis=1)

In [365]:
for_merge_not_supported_variant_evidence_percent_of_civic_df.to_csv('../for_merge_not_supported_variant_evidence_percent_of_civic_df.csv', index=False)

Summary Table 8: The table below shows the percentage of all CIViC evidence items that are associated with a Not Supported variant sub category. This will not add up to 100% since the evidence items can be associated with multiple variants.

In [366]:
not_supported_variant_evidence_percent_of_civic_df = not_supported_variant_evidence_percent_of_civic_df.set_index("Category")
not_supported_variant_evidence_percent_of_civic_df

,Percent of all CIViC Evidence Items
Category,
Expression,618 / 9914 (6.23%)
Epigenetic Modification,24 / 9914 (0.24%)
Fusion,1239 / 9914 (12.50%)
Protein Consequence,301 / 9914 (3.04%)
Gene Function,348 / 9914 (3.51%)
Rearrangements,136 / 9914 (1.37%)
Copy Number,64 / 9914 (0.65%)
Other,162 / 9914 (1.63%)
Genotypes Easy,23 / 9914 (0.23%)


In [367]:
civic_summary_table_8 = not_supported_variant_evidence_percent_of_civic_df

Summary Table 9: The table below shows the percentage of all evidence items associated with Not Supported variants that are associated with a variant sub category. 

In [368]:
not_supported_variant_evidence_percent_of_not_supported_df = not_supported_variant_evidence_percent_of_not_supported_df.set_index("Category")
not_supported_variant_evidence_percent_of_not_supported_df

,Percent of Not Supported Variant Evidence Items
Category,
Expression,618 / 4369 (14.15%)
Epigenetic Modification,24 / 4369 (0.55%)
Fusion,1239 / 4369 (28.36%)
Protein Consequence,301 / 4369 (6.89%)
Gene Function,348 / 4369 (7.97%)
Rearrangements,136 / 4369 (3.11%)
Copy Number,64 / 4369 (1.46%)
Other,162 / 4369 (3.71%)
Genotypes Easy,23 / 4369 (0.53%)


In [369]:
civic_summary_table_9 = not_supported_variant_evidence_percent_of_not_supported_df

Summary Table 10: The table below shows the percentage of evidence items associated with Not Supported variant sub categories that are accepted or submitted.

In [370]:
not_supported_variant_evidence_percent_evidence_df = not_supported_variant_evidence_percent_evidence_df.set_index("Category")
not_supported_variant_evidence_percent_evidence_df

,Percent of Accepted Evidence Items,Percent of Submitted Evidence Items
Category,,
Expression,342 / 618 (55.34%),276 / 618 (44.66%)
Epigenetic Modification,22 / 24 (91.67%),2 / 24 (8.33%)
Fusion,737 / 1239 (59.48%),502 / 1239 (40.52%)
Protein Consequence,191 / 301 (63.46%),110 / 301 (36.54%)
Gene Function,154 / 348 (44.25%),194 / 348 (55.75%)
Rearrangements,53 / 136 (38.97%),83 / 136 (61.03%)
Copy Number,27 / 64 (42.19%),37 / 64 (57.81%)
Other,60 / 162 (37.04%),102 / 162 (62.96%)
Genotypes Easy,13 / 23 (56.52%),10 / 23 (43.48%)


In [371]:
civic_summary_table_10 = not_supported_variant_evidence_percent_evidence_df

## Impact

In [372]:
not_supported_variant_impact_df

,Category,CIVIC Total Sum Impact Score,Average Impact Score per Variant,Average Impact Score per Evidence Item,Total Number Evidence Items,% Accepted Evidence Items,Total Number Variants
0,Expression,3618.00,9.397403,7.681529,618,55.34%,291
1,Epigenetic Modification,285.50,0.741558,0.606157,24,91.67%,15
2,Fusion,6600.75,17.144805,14.014331,1239,59.48%,308
3,Protein Consequence,2683.75,6.970779,5.697983,301,63.46%,128
4,Gene Function,1805.50,4.689610,3.833333,348,44.25%,90
5,Rearrangements,653.00,1.696104,1.386412,136,38.97%,52
6,Copy Number,225.00,0.584416,0.477707,64,42.19%,34
7,Other,673.50,1.749351,1.429936,162,37.04%,93
8,Genotypes Easy,195.00,0.506494,0.414013,23,56.52%,11
9,Genotypes Compound,117.50,0.305195,0.249469,7,57.14%,6


The bar graph below shows the relationship between the Not Suported variant sub category impact score and the sub category. Additionally, the colors illustrate the number of evidence items associated each sub category.

In [373]:
fig = px.bar(
    not_supported_variant_impact_df,
    x="Category",
    y="CIVIC Total Sum Impact Score",
    hover_data=["Total Number Evidence Items", not_supported_variant_impact_df["% Accepted Evidence Items"]],
    color="Total Number Evidence Items",
    labels={"CIVIC Total Sum Impact Score": "CIVIC Total Sum Impact Score"},
    text_auto=".1f",
    color_continuous_scale="geyser"
)
fig.update_traces(width=1)
fig.show()

In [374]:
fig.write_html("civic_ns_categories_impact_redgreen.html")

The scatterplot below shows the relationship between the Not Suported variant sub category impact score and the number of evidence items associated with variants in each sub category. Additionally, the sizes of the data point represent the number of variants in each sub category. 

In [375]:
fig2 = px.scatter(
    data_frame=not_supported_variant_impact_df,
    x="Total Number Evidence Items",
    y="CIVIC Total Sum Impact Score",
    size="Total Number Variants",
    size_max=40,
    text="Total Number Variants",
    color="Category",
    hover_data="% Accepted Evidence Items"
)
fig2.show()

In [376]:
fig2.write_html("civic_ns_categories_impact_scatterplot.html")

In [377]:
fig3= px.scatter(data_frame= not_supported_variant_impact_df,
                x= 'Total Number Variants',
                y= 'Average Impact Score per Evidence Item',
                size= 'Total Number Variants',
                size_max= 40,
                text= 'Total Number Variants',
                # color_discrete_sequence= Bold,
                color= 'Category',
                hover_data= ['% Accepted Evidence Items', 'Average Impact Score per Variant'])
fig3.show()